In [ ]:
!pip install pyarrow
!pip install -e orpheus_tts_pypi
!pip install python-dotenv

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import random
from ask_lisa import ask_lisa
from dotenv import load_dotenv

In [1]:
!pip install TTS
!pip install pyarrow

Looking in indexes: https://nexus.iisys.de/repository/ki-awz-pypi-group/simple, https://pypi.org/simple
Looking in indexes: https://nexus.iisys.de/repository/ki-awz-pypi-group/simple, https://pypi.org/simple


In [3]:
base_dir = "asr_bundestag_clean"
train = base_dir + "/train_nodev"
validate = base_dir + "/train_dev"
test = base_dir + "/test"

load_dotenv()

In [4]:
mapping = {}

with open(train + "/wav.scp", "r", encoding="utf-8") as f:
    for line in f:
        words = line.strip().split()
        mapping.update({words[0]: [words[1]]})

for i, (key, value) in enumerate(mapping.items()):
    if i >= 5:
        break
    print(key, ": ", value)

7535988_0_172_12 :  ['wavs/7535988_0_172_12.wav']
7536262_0_15_7 :  ['wavs/7536262_0_15_7.wav']
7546736_0_163_2 :  ['wavs/7546736_0_163_2.wav']
7511688_0_142_16 :  ['wavs/7511688_0_142_16.wav']
7535129_3_253 :  ['wavs/7535129_3_253.wav']


In [5]:
with open(train + "/text", "r", encoding="utf-8") as f:
    for line in f:
        words = line.strip().split(maxsplit=1)
        mapping[words[0]].append(words[1])

for i, (key, value) in enumerate(mapping.items()):
    if i >= 2:
        break
    print(key, ": ", value)

7535988_0_172_12 :  ['wavs/7535988_0_172_12.wav', 'das bedeutet jetzt dass die länder auf jeden fall daten erheben müssen']
7536262_0_15_7 :  ['wavs/7536262_0_15_7.wav', 'kein geschenk das ist verdient meine damen und herren']


In [6]:
word_count = {}

with open(train + "/text", "r", encoding="utf-8") as f:
    for line in f:
        count = len(line.strip().split()) - 1
        if count in word_count:
            word_count[count] = word_count[count] + 1
        else:
            word_count[count] = 1

counts = sorted(word_count.keys())
frequencies = [word_count[c] for c in counts]

plt.bar(counts, frequencies)
plt.xlabel("Wörter pro Transkript")
plt.ylabel("Häufigkeit")
plt.show()

In [7]:
filtered_mapping = {}
for key, value in mapping.items():
    count = len(value[1].split())
    if 70 < count:
        filtered_mapping.update({key: value})

mapping = filtered_mapping

for i, (key, value) in enumerate(mapping.items()):
    if i >= 2:
        break
    print(key, ": ", value)

print(len(mapping.items()))

7506424_2_69 :  ['wavs/7506424_2_69.wav', 'cdu csu fraktion frau präsidentin meine sehr geehrten damen und herren liebe kolleginnen und kollegen wo man am meisten drauf erpicht grad das bekommt man meistens nicht liebe kolleginnen und kollegen von den linken und den grünen dieses zitat von wilhelm busch passt perfekt zu ihren anträgen gern erläutere ich warum das so ist sie fordern dass die co kosten allein den vermieterinnen und vermietern aufgebürdet werden sollen und erhoffen sich hierdurch dass die energetische sanierung der gebäude angereizt wird']
7520734_0_99_1 :  ['wavs/7520734_0_99_1.wav', 'hat kann man davon ausgehen dass die regulierung der europäischen union egal ob das jetzt eine verordnung oder eine direktive wird dieselbe richtung gehen wird sie wird detaillierter sein sie wird bestimmte dinge nochmal ausführlicher auch sicher ausbuchstabieren ich halte das deutsche gesetz für eine gute vorbereitung und letzten endes gibt es den deutschen unternehmen da auch ein stück we

In [8]:
with open("system_prompt.txt", "r", encoding="utf-8") as f:
    system_prompt = f.read()

def generate_qa(context):
    qa_res = ask_lisa(system_prompt, context)

    try:
        qa_all = json.loads(qa_res)
    except json.JSONDecodeError:
        print("Antwort war kein gültiges JSON:", qa_res)
        return None, None, difficulty

    return qa_all.get("level_1", {}), qa_all.get("level_2", {}), qa_all.get("level_3", {})

In [9]:
generate_qa("es gibt zahlreiche nicht abklingende proteste es gibt initiativen es gibt self made projekte petitionen umfragen zeigen dass große teile der gesellschaft bereit sind jetzt wo wir hier reden haben wir vor dem bundestag eine begleitende kundgebung mit über zehn bewegungen und es gibt jetzt gerade redebeiträge und die zeigen ihnen auch wieder dass es nicht nur diese petition ist sondern dass es hunderttausende wenn nicht sogar millionen von menschen sind die tagtäglich immer wieder dafür kämpfen dass hier endlich was passiert im bereich der mobilitäts und verkehrswende zum schluss")

Response status code: 200


({'question': 'Was wird gefordert?', 'answer': 'Mobilitätswende'},
 {'question': 'Welche Formen des Protests gibt es?',
  'answer': 'Proteste, Initiativen, Petitionen, Kundgebungen'},
 {'question': 'Warum ist die breite Beteiligung an der Mobilitätswende entscheidend?',
  'answer': 'Sie zeigt gesellschaftlichen Druck und Legitimität für strukturelle Veränderungen'})

In [ ]:
from orpheus_tts import OrpheusModel
import wave
import os

from huggingface_hub import login
login(token=os.getenv("HUGGINGFACE_TOKEN"))

# Modell initialisieren
model = OrpheusModel(model_name ="canopylabs/3b-de-ft-research_release", max_model_len=2048)

def to_speech(text, key):
    # Audio generieren (streaming)
    syn_tokens = model.generate_speech(
        prompt=text,
        voice="default"  # Für deutsche Modelle, prüfe verfügbare Voices
    )
    
    # Als WAV speichern
    with wave.open("output.wav", "wb") as wf:
        wf.setnchannels(1)
        wf.setsampwidth(2)
        wf.setframerate(24000)
        
        for audio_chunk in syn_tokens:
            wf.writeframes(audio_chunk)


In [10]:
<<<<<<< LOCAL CELL DELETED >>>>>>>
tts = TTS("tts_models/de/thorsten/vits", progress_bar=False)
def to_speech(text):
    tts.tts_to_file(text=text, file_path="output.wav")
    with open("output.wav", "rb") as wav_file:
        return wav_file.read()

 > tts_models/de/thorsten/vits is already downloaded.
 > Using model: vits
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024


In [11]:
keys = list(mapping.keys())

context_audio = []
context_text = []

q_1_audio = []
q_1_text = []
a_1_text = []

q_2_audio = []
q_2_text = []
a_2_text = []

q_3_audio = []
q_3_text = []
a_3_text = []

train_size = 2 #1200
val_size = 1 #150
test_size = 1 #150

In [12]:
def generate_sample(key):
    context = mapping[key][1]
    context_text.append(context)
    with open(base_dir + "/" + mapping[key][0], "rb") as wav_file:
        context_audio.append(wav_file.read())
    
    level_1, level_2, level_3 = generate_qa(mapping[key][1])

    q_1 = level_1.get("question")
    q_1_text.append(q_1)
    q_1_audio.append(to_speech(q_1))
    a_1_text.append(level_1.get("answer"))

    q_2 = level_2.get("question")
    q_2_text.append(q_2)
    q_2_audio.append(to_speech(q_2))
    a_2_text.append(level_2.get("answer"))

    q_3 = level_3.get("question")
    q_3_text.append(q_3)
    q_3_audio.append(to_speech(q_3))
    a_3_text.append(level_3.get("answer"))

In [13]:
def to_df():
    return pd.DataFrame({
    "context_text": context_text,
    "context_audio": context_audio,
    "q_1_text": q_1_text,
    "q_1_audio": q_1_audio,
    "a_1_text": a_1_text,
    "q_2_text": q_2_text,
    "q_2_audio": q_2_audio,
    "a_2_text": a_2_text,
    "q_3_text": q_3_text,
    "q_3_audio": q_3_audio,
    "a_3_text": a_3_text
})

In [14]:
def clear_arrays():
    context_audio.clear()
    context_text.clear()
    q_1_audio.clear()
    q_1_text.clear()
    a_1_text.clear()
    q_2_audio.clear()
    q_2_text.clear()
    a_2_text.clear()
    q_3_audio.clear()
    q_3_text.clear()
    a_3_text.clear()

In [15]:
# Trainingsdatensatz
clear_arrays()

for i in range(train_size):
    key = random.choice(keys)
    generate_sample(key)
    keys.remove(key)

train_df = to_df()
train_df.to_parquet("train.parquet", engine="pyarrow")

Response status code: 200
 > Text splitted to sentences.
['Warum ist Mundschutz nicht notwendig?']


[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.


 > Processing time: 0.4472842216491699
 > Real-time factor: 0.15530702140596178
 > Text splitted to sentences.
['Was wird dem BMG vorgeworfen?']
 > Processing time: 0.3825244903564453
 > Real-time factor: 0.1517899692693568
 > Text splitted to sentences.
['Welche Konsequenzen hatte die BMG-Einschätzung?']
vɛlçə kɔnzeːkvuːeːntsən hatə diː bmk aɪ̯nʃɛtsʊŋ?
 [!] Character '̯' not found in the vocabulary. Discarding it.
 > Processing time: 0.40167856216430664
 > Real-time factor: 0.11122428540942036
Response status code: 200
 > Text splitted to sentences.
['Worum geht es?']
 > Processing time: 0.22657036781311035
 > Real-time factor: 0.1291858866952597
 > Text splitted to sentences.
['Wird ein einheitlicher Ansprechpartner bei Telegram eingerichtet?']
 > Processing time: 0.45662546157836914
 > Real-time factor: 0.10628052089810675
 > Text splitted to sentences.
['Warum ist ein einheitlicher Ansprechpartner bei Telegram wichtig?']
 > Processing time: 0.43591952323913574
 > Real-time factor: 

In [16]:
# Validierungsdatensatz
clear_arrays()

for i in range(val_size):
    key = random.choice(keys)
    generate_sample(key)
    keys.remove(key)

val_df = to_df()
val_df.to_parquet("validate.parquet", engine="pyarrow")

Response status code: 200
 > Text splitted to sentences.
['Wer betonte die Aussage?']
 > Processing time: 0.6740074157714844
 > Real-time factor: 0.27120188901024145
 > Text splitted to sentences.
['Warum sind mehr Investitionen nötig?']
 > Processing time: 0.35756659507751465
 > Real-time factor: 0.1226715120341548
 > Text splitted to sentences.
['Wie unterscheidet sich die Position von der CDU/CSU-Programmatik?']
 > Processing time: 0.5476295948028564
 > Real-time factor: 0.12378252178738504


In [17]:
clear_arrays()

# Testdatensatz
for i in range(test_size):
    key = random.choice(keys)
    generate_sample(key)
    keys.remove(key)

test_df = to_df()
test_df.to_parquet("test.parquet", engine="pyarrow")

Response status code: 200
 > Text splitted to sentences.
['Welche Technik wird erwähnt?']
 > Processing time: 0.2572038173675537
 > Real-time factor: 0.10446773086049511
 > Text splitted to sentences.
['Wofür soll digitale Technik eingesetzt werden?']
 > Processing time: 0.36066436767578125
 > Real-time factor: 0.09705928172294201
 > Text splitted to sentences.
['Wie steht die Bundesrepublik im Einsatz digitaler Lernsoftware zur Überwindung von Lernrückständen?']
 > Processing time: 0.7221963405609131
 > Real-time factor: 0.11647476089356446
